## 1. Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Import data
harmonized25_74 = pd.read_csv('../data/other_tsv/teilm022.tsv', sep='\t')
harmonized25_74.head()

,"s_adj,age,sex,unit,geo\time",2019M09,2019M10,2019M11,2019M12,2020M01,2020M02,2020M03,2020M04,2020M05,2020M06,2020M07,2020M08
0,"SA,Y25-74,F,PC_ACT,AT",3.7,3.8,3.7,3.7,3.7,3.6,3.7,4.0,4.2,4.1,3.9,:
1,"SA,Y25-74,F,PC_ACT,BE",4.3,4.1,3.9,3.9,4.1,4.2,4.4,4.5,4.4,4.4,4.4,:
2,"SA,Y25-74,F,PC_ACT,BG",3.5,3.6,3.6,3.5,3.8,3.6,3.6,4.2,4.0,3.9,3.9,:
3,"SA,Y25-74,F,PC_ACT,CH",4.0,3.8,3.6,3.7,4.0,4.1,4.1,4.0,4.1,4.2,:,:
4,"SA,Y25-74,F,PC_ACT,CY",7.0,6.8,6.6,6.3,6.0,5.6,5.9,7.3,7.3,6.1,5.3,:


## 2. Data Wrangling

For better accessibility of columns, **column names** are harmonized. Furthermore multiple information in categorical columns is assigned to **individual columns** for deeper analysis.

In [3]:
# Strip column names of white spaces
harmonized25_74.columns = (harmonized25_74.columns.str.strip()
                                                  .str.replace('/', '_'))
harmonized25_74.columns 

Index(['s_adj,age,sex,unit,geo\time', '2019M09', '2019M10', '2019M11',
       '2019M12', '2020M01', '2020M02', '2020M03', '2020M04', '2020M05',
       '2020M06', '2020M07', '2020M08'],
      dtype='object')

In [4]:
# Split first column
# 's_adj' means 'seasonally adjusted form'
harmonized25_74[['s_adj', 'age','sex','unit', 'geo_time']] = harmonized25_74.iloc[:,0].str.split(',', expand=True)
harmonized25_74.head()

,"s_adj,age,sex,unit,geo\time",2019M09,2019M10,2019M11,2019M12,2020M01,2020M02,2020M03,2020M04,2020M05,2020M06,2020M07,2020M08,s_adj,age,sex,unit,geo_time
0,"SA,Y25-74,F,PC_ACT,AT",3.7,3.8,3.7,3.7,3.7,3.6,3.7,4.0,4.2,4.1,3.9,:,SA,Y25-74,F,PC_ACT,AT
1,"SA,Y25-74,F,PC_ACT,BE",4.3,4.1,3.9,3.9,4.1,4.2,4.4,4.5,4.4,4.4,4.4,:,SA,Y25-74,F,PC_ACT,BE
2,"SA,Y25-74,F,PC_ACT,BG",3.5,3.6,3.6,3.5,3.8,3.6,3.6,4.2,4.0,3.9,3.9,:,SA,Y25-74,F,PC_ACT,BG
3,"SA,Y25-74,F,PC_ACT,CH",4.0,3.8,3.6,3.7,4.0,4.1,4.1,4.0,4.1,4.2,:,:,SA,Y25-74,F,PC_ACT,CH
4,"SA,Y25-74,F,PC_ACT,CY",7.0,6.8,6.6,6.3,6.0,5.6,5.9,7.3,7.3,6.1,5.3,:,SA,Y25-74,F,PC_ACT,CY


In [5]:
# Reorder columns
harmonized25_74_copy = harmonized25_74[['s_adj', 'age', 'sex', 'unit', 'geo_time', '2019M09', '2019M10', '2019M11', '2019M12', '2020M01', '2020M02', '2020M03', '2020M04', '2020M05','2020M06', '2020M07', '2020M08']]
harmonized25_74_copy.head()

,s_adj,age,sex,unit,geo_time,2019M09,2019M10,2019M11,2019M12,2020M01,2020M02,2020M03,2020M04,2020M05,2020M06,2020M07,2020M08
0,SA,Y25-74,F,PC_ACT,AT,3.7,3.8,3.7,3.7,3.7,3.6,3.7,4.0,4.2,4.1,3.9,:
1,SA,Y25-74,F,PC_ACT,BE,4.3,4.1,3.9,3.9,4.1,4.2,4.4,4.5,4.4,4.4,4.4,:
2,SA,Y25-74,F,PC_ACT,BG,3.5,3.6,3.6,3.5,3.8,3.6,3.6,4.2,4.0,3.9,3.9,:
3,SA,Y25-74,F,PC_ACT,CH,4.0,3.8,3.6,3.7,4.0,4.1,4.1,4.0,4.1,4.2,:,:
4,SA,Y25-74,F,PC_ACT,CY,7.0,6.8,6.6,6.3,6.0,5.6,5.9,7.3,7.3,6.1,5.3,:


The dataset does not contain any **null values**.

In [6]:
# Check data types
harmonized25_74_copy.dtypes

s_adj        object
age          object
sex          object
unit         object
geo_time     object
2019M09     float64
2019M10     float64
2019M11     float64
2019M12     float64
2020M01      object
2020M02      object
2020M03      object
2020M04      object
2020M05      object
2020M06      object
2020M07      object
2020M08      object
dtype: object

In [7]:
# Investigate object columns that should be numeric
harmonized25_74_copy['2020M01'].value_counts()

2.9      8
5.4      6
2.8      5
7.1      4
4.4      4
6.6      4
5.1      4
3.7      4
5.8      4
2.4      4
5.5      4
4.1      3
5.7      3
2.7      3
6.1      3
:        3
3.9      3
2.5      2
3.5      2
4.0      2
4.6      2
5.3      2
6.0      2
4.7      2
3.2      2
3.8      2
5.6      2
10.7     1
1.6      1
3.0      1
2.6      1
6.5      1
19.0     1
4.8      1
5.9      1
12.4     1
1.8      1
15.3     1
12.5     1
7.5      1
5.2      1
6.8      1
3.5 e    1
6.2      1
3.2 e    1
10.8     1
3.6      1
2.8 e    1
2.1      1
4.5      1
14.5     1
2.3      1
4.9      1
Name: 2020M01, dtype: int64

Since only few values have a letter, the whole dataframe is set to **numeric** in the month columns.

In [9]:
cols = ['2019M09','2019M10', '2019M11', '2019M12', '2020M01', '2020M02', '2020M03', '2020M04', '2020M05', '2020M06', '2020M07', '2020M08']
harmonized25_74_copy[cols] = harmonized25_74_copy[cols].apply(pd.to_numeric, errors='coerce')
harmonized25_74_copy.head()


C:\Users\yuliz\anaconda3\envs\matrix\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,s_adj,age,sex,unit,geo_time,2019M09,2019M10,2019M11,2019M12,2020M01,2020M02,2020M03,2020M04,2020M05,2020M06,2020M07,2020M08
0,SA,Y25-74,F,PC_ACT,AT,3.7,3.8,3.7,3.7,3.7,3.6,3.7,4.0,4.2,4.1,3.9,NaN
1,SA,Y25-74,F,PC_ACT,BE,4.3,4.1,3.9,3.9,4.1,4.2,4.4,4.5,4.4,4.4,4.4,NaN
2,SA,Y25-74,F,PC_ACT,BG,3.5,3.6,3.6,3.5,3.8,3.6,3.6,4.2,4.0,3.9,3.9,NaN
3,SA,Y25-74,F,PC_ACT,CH,4.0,3.8,3.6,3.7,4.0,4.1,4.1,4.0,4.1,4.2,NaN,NaN
4,SA,Y25-74,F,PC_ACT,CY,7.0,6.8,6.6,6.3,6.0,5.6,5.9,7.3,7.3,6.1,5.3,NaN


## 3. Export Cleaned Data File

In [10]:
harmonized25_74_copy.to_csv('teilm022_cleaned.csv', index = False)